In [1]:
import pickle
import sys
import inspect
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from rdkit import Chem, DataStructs, RDLogger
from rdkit.Chem import AllChem

from rad.construction import getGraphs
from rad.traversal import traverseHNSW
from utils.paths import scores_path

# Disable rdkit error logging to keep output clean
RDLogger.DisableLog('rdApp.*') 

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<int, std::allocator<int> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<double, std::allocator<double> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, std::allocator<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > > > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<unsigned long, std::allocator<unsigned long> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<std::vector<double, std::allocator<double> >, std::allocator<s

### Load the DUDEZ Goldilocks dataset

In [2]:
input_pickle = "goldilocks_smiles.pkl"
with open(input_pickle, 'rb') as f:
    dudez_data = pickle.load(f)

In [3]:
print(f"Loaded {len(dudez_data)} molecules from {input_pickle}.")

Loaded 132884 molecules from goldilocks_smiles.pkl.


### Shuffle the order of the dataset

In [4]:
import random

In [5]:
items = list(dudez_data.items())
random.seed(42)
random.shuffle(items)
dudez_data = dict(items) # rebuilds dict in new order

### Set parameters for fingerprints and generate them

In [6]:
FP_LENGTH = 1024
FP_RADIUS = 2

In [7]:
failed_smiles = []  # List to store failed SMILES
successful_count = 0  # Counter for successful molecules

In [8]:
dudez_fps = [] ## To store molecular fingerprints in a compact binary form (packed bits).
node_id = 0 ## A unique identifier for each molecule (node).

# Store a mapping from node_id -> zID
id_to_zid = []

for zid in tqdm(dudez_data, total=len(dudez_data), desc="Generating Fingeprints"):
    smi = dudez_data[zid] ## SMILES string for the molecule.

    # Some smiles will fail molecule generation. We just skip them
    mol = Chem.MolFromSmiles(smi) ## The RDKit library converts the SMILES string into a molecular object
    if mol is None:
        ## Log failed SMILES
        failed_smiles.append(smi)
        continue
    
    # If successful, process as usual
    successful_count += 1
    # Convert rdkit bit vect fingerprint to numpy array
    arr = np.zeros((1,), dtype=np.uint8)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=FP_RADIUS, nBits=FP_LENGTH) ## Generate Molecular Fingerprints
    DataStructs.ConvertToNumpyArray(fp, arr) ## Convert Morgan Fingerprints to NumPy Array

    # IMPORTANT: Make sure to pack bit fingerprints - it vastly speeds up HNSW construction
    dudez_fps.append(np.packbits(arr)) 
    ## Groups every 8 bits into a single byte.
    ## Reduces the length of the array by a factor of 8 (e.g., from 1024 bits to 128 bytes).
    
    id_to_zid.append(zid) # Record that node_id corresponds to this zID
    
    node_id += 1 ## Every molecule gets a unique identifier

dudez_fps = np.array(dudez_fps) ## Convert Fingerprints List to NumPy Array
# Output results
print(f"Total SMILES processed: {len(dudez_data)}")
print(f"Successful molecule generation: {successful_count}")
print(f"Failed SMILES count: {len(failed_smiles)}")
print("Failed SMILES examples:", failed_smiles[:10])  # Print a few failed SMILES

Generating Fingeprints: 100%|██████████| 132884/132884 [00:27<00:00, 4792.02it/s]


Total SMILES processed: 132884
Successful molecule generation: 132881
Failed SMILES count: 3
Failed SMILES examples: ['=C(NCC1=CC=CN=C1)[C@H]1[C@@H]2C[N@@H+](C[C@@H]3C[C@H]4C=C[C@@H]3C4)C[C@@H]21', '1=CC=C(CCC[N@@H+]2C[C@@H]3CC[C@H](C2)N(C[C@H]2[C@@H]4C[NH2+]C[C@H]24)C3)C=C1', '[C@H]2[C@H]3C=C[C@H](C3)[C@@H]2C(=O)N1NC1=CC=C([N+](=O)[O-])C=C1[N+](=O)[O-]']


### Set parameters for HNSW and construct it

In [10]:
EF_CONSTRUCTION = 400 ## graph_quality
M = 16 ## max_neighbours_node

In [11]:
hnsw_layer_graphs, hnsw_index = getGraphs(dudez_fps, ef_construction=EF_CONSTRUCTION, M=M)
element_levels = hnsw_index["element_levels"]
max_level = hnsw_index["max_level"]

for lvl in range(max_level + 1):
    num_nodes_in_layer = np.sum(element_levels >= lvl)
    print(f"Number of nodes in layer {lvl}: {num_nodes_in_layer}")

HNSW Construction time: 23.752772569656372 seconds


Formatting hnswlib Neighbor Data: 132881it [00:00, 462359.43it/s]
Constructing graph_tool graphs: 100%|█████████▉| 132376/132881 [00:02<00:00, 56287.04it/s]

Adding batch 0/0 to graphs


Constructing graph_tool graphs: 100%|██████████| 132881/132881 [00:05<00:00, 23178.67it/s]

Number of nodes in layer 0: 132881
Number of nodes in layer 1: 8217
Number of nodes in layer 2: 491
Number of nodes in layer 3: 30
Number of nodes in layer 4: 1


### Score_fn using docked scores

In [12]:
import json

In [13]:
NUM_TO_TRAVERSE = 10000 # Maximum number of molecules to score

In [14]:
# 1) Load all JSON into one dictionary
score_data = {}
for i in range(7):
    filename = scores_path(f"goldilocks_scores{i}.json")
    with open(filename, "r") as f:
        data = json.load(f)
        # Each entry is like "ZINC000170632481": ["CC(C)C1=...", -7.6]
        for zid, (smiles, score) in data.items():
            score_data[zid] = (smiles, score)

In [15]:
# 2) Define the score function
scores_by_node = {}  # global or external dict to store node info
def score_fn(node_id):
    ligand_id = id_to_zid[node_id] 
    if ligand_id in score_data:
        smiles, docking_score = score_data[ligand_id]
        scores_by_node[ligand_id] = [node_id, smiles, docking_score]
        return docking_score
    else:
        return np.inf

In [16]:
traversed_nodes = traverseHNSW(hnsw_layer_graphs, score_fn, NUM_TO_TRAVERSE)

Traversing HNSW:  97%|█████████▋| 970/1000 [00:00<00:00, 344961.41it/s]


### Save the new traversed results

In [17]:
with open("radsmina1krandom.json", "w") as f:
    json.dump(scores_by_node, f, indent=2)